# Self study 2


In this self-study we build an index that supports Boolean search over the web pages that you crawl with the crawler from the 1st self study. You can continue to just extract the titles of the web-pages you crawl, or you can be more adventurous and look at the whole text that you get from the .get_text() method of a BeautifulSoup parser. In either case, the collection of texts from the crawled web-pages is you corpus. You should then:

- construct the vocabulary of terms for your corpus
- build an 'inverted' index for your vocabulary
- implement Boolean search for your index (perhaps only for a limited set of Boolean queries)

In [1]:
# Some things already used in self study 1:
import requests
from bs4 import BeautifulSoup
from time import sleep
from urllib.robotparser import RobotFileParser
from urllib.parse import urlparse
from datetime import datetime, timedelta
from itertools import count
import pandas as pd
import numpy as np
import random
import nltk
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
import codecs


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


What is most useful for you depends on which websites you crawl. It is not essential for the exercise that the stemming always is the best possible ...!

In [2]:
index_frame = pd.read_csv('index.csv')
index_arr = index_frame.to_dict('records')

index_arr = index_arr[0]
index_arr.pop('Unnamed: 0')

for key in index_arr.keys():
    print(f"{key}: {index_arr[key]}")

https://www.tv2.dk: TV 2 - bedst pÃ¥ breaking og live
https://www.bt.dk: B.T. Nyheder | Læs nyhederne på bt.dk
https://www.berlingskemedia.dk/handelsbetingelser: Generelle handelsbetingelser
https://www.bt.dk/cookiedeklaration: Cookiedeklaration
https://www.dr.dk: DR | Nyheder - Breaking - TV - Radio
https://www.bt.dk/kongehusetbagkulissen: Kongehuset bag kulissen
https://www.weekendavisen.dk/mine-sider/kundeservice: Mine sider | Weekendavisen
https://www.weekendavisen.dk/: Weekendavisen | Weekendavisen
https://www.berlingske.dk/mine-sider/kundeservice: Nyheder og seneste nyt fra Berlingske - Berlingske.dk
https://www.weekendavisen.dk/titusinde: Podcast: Lone Frank & de titusinde sjæle | Weekendavisen
https://www.weekendavisen.dk/en-ny-ladning-roeverhistorier: En ny ladning Røverhistorier | Weekendavisen
https://www.bt.dk/mine-sider/kundeservice: BT | Mine sider
https://www.dr.dk/om-dr/vilkaar-paa-drdk: Vilkår på dr.dk | Om DR | DR
https://www.dr.dk/om-dr/job: Job | DR
https://www.euro

In [3]:
for key in index_arr.keys():
    print(f"{key}: {index_arr[key]}")

https://www.tv2.dk: TV 2 - bedst pÃ¥ breaking og live
https://www.bt.dk: B.T. Nyheder | Læs nyhederne på bt.dk
https://www.berlingskemedia.dk/handelsbetingelser: Generelle handelsbetingelser
https://www.bt.dk/cookiedeklaration: Cookiedeklaration
https://www.dr.dk: DR | Nyheder - Breaking - TV - Radio
https://www.bt.dk/kongehusetbagkulissen: Kongehuset bag kulissen
https://www.weekendavisen.dk/mine-sider/kundeservice: Mine sider | Weekendavisen
https://www.weekendavisen.dk/: Weekendavisen | Weekendavisen
https://www.berlingske.dk/mine-sider/kundeservice: Nyheder og seneste nyt fra Berlingske - Berlingske.dk
https://www.weekendavisen.dk/titusinde: Podcast: Lone Frank & de titusinde sjæle | Weekendavisen
https://www.weekendavisen.dk/en-ny-ladning-roeverhistorier: En ny ladning Røverhistorier | Weekendavisen
https://www.bt.dk/mine-sider/kundeservice: BT | Mine sider
https://www.dr.dk/om-dr/vilkaar-paa-drdk: Vilkår på dr.dk | Om DR | DR
https://www.dr.dk/om-dr/job: Job | DR
https://www.euro

In [4]:
dstemmer=SnowballStemmer("danish")
a_file = codecs.open("stopord.txt", "r", "utf-8")
file_contents = a_file.read()
contents_split = file_contents.splitlines()
stop_words = []
for word in contents_split:
    if dstemmer.stem(word) not in stop_words:
        stop_words.append(dstemmer.stem(word))
extra_words = ["|", ",", ".", "-", "!"]
for word in extra_words:
    stop_words.append(word)

for word in stop_words:
    print(word)
a_file.close()

ad
af
aldr
alen
all
alligevel
alt
altid
and
andr
at
bag
bar
beg
bl.a.
bland
blev
bliv
burd
bør
ca.
da
de
dem
den
der
dereft
derfor
derfra
deri
dermed
derpå
derved
det
dig
din
dis
dit
dog
du
eft
egen
ej
ell
en
end
endnu
ene
enest
enhv
ens
ent
er
et
f.eks.
far
fem
fik
fir
fler
flest
for
foran
fordi
fra
fx
få
får
før
først
gennem
gjord
gjort
god
godt
gør
ham
han
har
havd
hav
hej
hel
helt
hen
hend
henov
her
hereft
heri
hermed
herpå
hos
hun
hvad
hvem
hver
hvilk
hvis
hvor
hvordan
hvoreft
hvorfor
hvorfra
hvorh
hvori
hvorimod
hvornår
hvorved
i
igen
igennem
ikk
imellem
imen
imod
ind
indtil
ing
int
ja
jeg
jer
jo
kan
kom
kun
lad
lang
lav
lidt
lig
ligesom
lil
læng
man
mand
mang
med
meg
mellem
men
mer
mest
mig
min
mindr
mindst
mit
mod
må
måsk
ned
nej
nem
ni
nog
nogensind
nogl
nok
nu
ny
nyt
når
nær
næst
og
også
okay
om
omkring
op
os
ott
over
overalt
pga.
på
sam
se
sek
selv
selvom
sen
ser
ses
sid
sig
sin
sit
skal
skul
som
stad
stor
syn
synt
syv
så
sådan
såled
tag
tem
thi
ti
tid
til
tilbag
tit
to
tre


In [29]:
stemmed_index = {}
for key in index_arr.keys():
    try:
        tokens = nltk.word_tokenize(index_arr[key])
        local_tokens = []
        for token in tokens:
            found_tokens = []
            if token not in stop_words:
                if token not in found_tokens:
                    local_tokens.append({'token' : dstemmer.stem(token), 'count' : 1})
                    found_tokens.append(token)
                else:
                    for token_dict in local_tokens:
                        if token_dict['token'] == token:
                            token_dict['count'] += 1


        stemmed_index[key] = local_tokens
    except:
        continue

#used to give numbers
numbered_index = {}
i = 1
for url in stemmed_index.keys():
    stemmed_index[url] = {'tokens' : stemmed_index[url], 'id' : i}
    numbered_index[i] = url
    i += 1
for key in stemmed_index.keys():
    print(f"{key}: {stemmed_index[key]['tokens']} [{stemmed_index[key]['id']}]")

https://www.tv2.dk: [{'token': 'tv', 'count': 1}, {'token': '2', 'count': 1}, {'token': 'bedst', 'count': 1}, {'token': 'pã¥', 'count': 1}, {'token': 'breaking', 'count': 1}, {'token': 'liv', 'count': 1}] [1]
https://www.bt.dk: [{'token': 'b.t', 'count': 1}, {'token': 'nyhed', 'count': 1}, {'token': 'læs', 'count': 1}, {'token': 'nyhed', 'count': 1}, {'token': 'bt.dk', 'count': 1}] [2]
https://www.berlingskemedia.dk/handelsbetingelser: [{'token': 'generel', 'count': 1}, {'token': 'handelsbeting', 'count': 1}] [3]
https://www.bt.dk/cookiedeklaration: [{'token': 'cookiedeklaration', 'count': 1}] [4]
https://www.dr.dk: [{'token': 'dr', 'count': 1}, {'token': 'nyhed', 'count': 1}, {'token': 'breaking', 'count': 1}, {'token': 'tv', 'count': 1}, {'token': 'radio', 'count': 1}] [5]
https://www.bt.dk/kongehusetbagkulissen: [{'token': 'kongehus', 'count': 1}, {'token': 'kulis', 'count': 1}] [6]
https://www.weekendavisen.dk/mine-sider/kundeservice: [{'token': 'min', 'count': 1}, {'token': 'sid',

In [48]:
simple_inverted_index = {}

for url in stemmed_index.keys():
    for token_dict in stemmed_index[url]['tokens']:
        token = token_dict['token']
        if token in simple_inverted_index.keys():
            if stemmed_index[url]['id'] not in simple_inverted_index[token]:
                simple_inverted_index[token].append({'posting': stemmed_index[url]['id'], 'frequency': token_dict['count']})
        else:
            simple_inverted_index[token] = [{'posting': stemmed_index[url]['id'], 'frequency': token_dict['count']}]

for key in simple_inverted_index.keys():
    print(f"{key}: {simple_inverted_index[key]}")

for key, value in simple_inverted_index.items():
    for token_dict in value:
        if (token_dict['frequency'] >= 2):
            print(key)
    

In [40]:
#Single word search

def convert_postings_to_urls(found_postings):
    found_urls = []
    for posting in found_postings:
        found_urls.append(numbered_index[posting])
    return found_urls

def get_postings(token_array):
    arr = []
    for token_dict in token_array:
        arr.append(token_dict['posting'])
    return arr


def single_word_search(search_word, want_postings = False):
    found_postings = []
    for token in simple_inverted_index.keys():
        if (token == search_word):
            found_postings = get_postings(simple_inverted_index[token])
    if want_postings:
        return found_postings
    if found_postings:
        return convert_postings_to_urls(found_postings)
    return []
    

for url in single_word_search('senest'):
    print(url)

https://www.berlingske.dk/mine-sider/kundeservice
https://www.berlingske.dk/berlingske-abonnementer?flow=bdk_print_checkout&product=kombi&offerId=8069.7601&campaignId=CAMP-BER-ID0227&bem_channel=own_banners&bem_placement=bdk&bem_type=red_teaser_artikel&bem_campaign=be_kombi_08_2022


In [42]:
#Search

def OR_merge(found_urls_one, found_urls_two):
    return list(set(found_urls_one + found_urls_two))

def AND_merge(found_urls_one, found_urls_two):
    return list(set(found_urls_one).intersection(set(found_urls_two)))

def AND_NOT_merge(found_urls_one, found_urls_two):
    a = set(found_urls_one)
    b = set(found_urls_two)    
    return list(a-b)

def Search(word_one, search_type, word_two):
    stemmed_word_one = dstemmer.stem(word_one)
    found_urls_one = single_word_search(stemmed_word_one, want_postings=True)
    stemmed_word_two = dstemmer.stem(word_two)
    found_urls_two = single_word_search(stemmed_word_two, want_postings=True)
    if(search_type == 'AND'):
        return AND_merge(found_urls_one, found_urls_two)
    elif(search_type == 'OR'):
        return OR_merge(found_urls_one, found_urls_two)
    elif(search_type == 'AND NOT'):
        return AND_NOT_merge(found_urls_one, found_urls_two)
    else:
        return ['Invalid search type']

for url in convert_postings_to_urls(Search('Universitet', 'OR', 'Kat')):
    print(url)

https://www.bio.aau.dk/om-instituttet/organisation
https://www.aau.dk
https://www.studerende.aau.dk
https://www.search.aau.dk?site=www.bio.aau.dk&locale=da&mobile=false
https://www.studerende.aau.dk/vejledning
https://www.alumni.aau.dk/
https://www.its.aau.dk/support
https://www.stillinger.aau.dk/
https://www.aau.dk/uddannelser/su/studieaktivitetskontrol
https://www.aau.dk/pressen
https://www.tech.aau.dk/forskning/god-videnskabelig-praksis
https://www.studerende.aau.dk/arrangementer
https://www.bio.aau.dk/om-instituttet/adjungerede-professorer-og-lektorer
https://www.tech.aau.dk/om-tech/pressekontakter
https://www.aau.dk/om-aau/organisation/faelles-service-og-innovation
https://www.search.aau.dk?site=www.its.aau.dk&locale=da&mobile=false
https://www.match.aau.dk/
https://www.search.aau.dk?site=www.search.aau.dk&locale=da&mobile=false
https://www.tech.aau.dk/om-tech/kontakt
https://www.search.aau.dk?site=www.aau.dk&locale=da&mobile=false
https://www.aau.dk/samarbejde/kontakt-os
https://

In [ ]:

def OR_merge(found_urls_one, found_urls_two):
    return list(set(found_urls_one + found_urls_two))

def AND_merge(found_urls_one, found_urls_two):
    return list(set(found_urls_one).intersection(set(found_urls_two)))

def AND_NOT_merge(found_urls_one, found_urls_two):
    a = set(found_urls_one)
    b = set(found_urls_two)    
    return list(a-b)

def Search(word_one, search_type, word_two):
    stemmed_word_one = dstemmer.stem(word_one)
    found_urls_one = single_word_search(stemmed_word_one, want_postings=True)
    stemmed_word_two = dstemmer.stem(word_two)
    found_urls_two = single_word_search(stemmed_word_two, want_postings=True)
    if(search_type == 'AND'):
        return AND_merge(found_urls_one, found_urls_two)
    elif(search_type == 'OR'):
        return OR_merge(found_urls_one, found_urls_two)
    elif(search_type == 'AND NOT'):
        return AND_NOT_merge(found_urls_one, found_urls_two)
    else:
        return ['Invalid search type']

for url in convert_postings_to_urls(Search('Universitet', 'OR', 'Kat')):
    print(url)